In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
0,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,Female,45,Mumbai,,Able to live my life comfortably on my own ter...,My growing up years were very nondescript. I h...,"Was married earlier, legally divorced",Yes,Between 20-24 Lakhs,I am a corporate lawyer and company secretary....,...,Masters,I am a chilled happy go lucky person who value...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,,
379,Male,32,Delhi,,Supportive family,"""Adventurous soul seeking a partner in crime f...",Never married,No,Between 12-15 Lakhs,It professional,...,Masters,"""Adventurous soul seeking a partner in crime f...",I give consent,Hindu,Yes,It will not work for me,Yes,Instagram,,
380,Male,44,Pune / Mumbai,,Be simple ...its interesting,Grow with flow,"Separated, filed for a divorce",Yes,Between 24-30 Lakhs,The story teller,...,Phd and equivalent,Make a moment...that will cherish,I give consent,Spiritual,Yes,Yes I am,Yes,Instagram,,
381,Female,49,Faridabad,,"Well educated, being Financially independent ...","Were full of drama but there was love too , o...",Widow/ Widower,Yes,Between 24-30 Lakhs,Social entrepreneur,...,Undergraduate,"I am a well recognized social worker , worki

In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
2,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,Female,45,Mumbai,,Able to live my life comfortably on my own ter...,My growing up years were very nondescript. I h...,"Was married earlier, legally divorced",Yes,Between 20-24 Lakhs,I am a corporate lawyer and company secretary....,...,Masters,I am a chilled happy go lucky person who value...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,,
381,Male,32,Delhi,,Supportive family,"""Adventurous soul seeking a partner in crime f...",Never married,No,Between 12-15 Lakhs,It professional,...,Masters,"""Adventurous soul seeking a partner in crime f...",I give consent,Hindu,Yes,It will not work for me,Yes,Instagram,,
382,Male,44,Pune / Mumbai,,Be simple ...its interesting,Grow with flow,"Separated, filed for a divorce",Yes,Between 24-30 Lakhs,The story teller,...,Phd and equivalent,Make a moment...that will cherish,I give consent,Spiritual,Yes,Yes I am,Yes,Instagram,,
383,Female,49,Faridabad,,"Well educated, being Financially independent ...","Were full of drama but there was love too , o...",Widow/ Widower,Yes,Between 24-30 Lakhs,Social entrepreneur,...,Undergraduate,"I am a well recognized social worker , worki

In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
380,Female,45,Mumbai,"Was married earlier, legally divorced"
381,Male,32,Delhi,Never married
382,Male,44,Pune / Mumbai,"Separated, filed for a divorce"
383,Female,49,Faridabad,Widow/ Widower


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Legally Separated,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
380,0,0,0,0,0,1,0
381,0,1,0,0,0,0,0
382,0,0,1,0,0,0,0
383,0,0,0,0,0,0,1


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
380,Female,45,Mumbai,"Was married earlier, legally divorced"
381,Male,32,Delhi,Never married
382,Male,44,Pune / Mumbai,"Separated, filed for a divorce"
383,Female,49,Faridabad,Widow/ Widower


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
380,Female,45,Mumbai,"Was married earlier, legally divorced"
381,Male,32,Delhi,Never married
382,Male,44,Pune / Mumbai,"Separated, filed for a divorce"
383,Female,49,Faridabad,Widow/ Widower


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(383, 145)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 252 316  11 108 372 272 266 342  45 165 313  25 331 278
  320  41 330 343 332 326 245 247 290 156 351  48 261  81  24 311 229  26
  358 309 139 181 256  97 118 363 333 123 354 288  51 122  30 128 295  54
  110 376 262  86  84 136 349  19 381 231 260  75  55 259 325 274 307 114
  310 336 365 308 200  52  42 183 375 287 265 355 174  27   1   6 353 105
   50  73  18  32 100 378 228  29 327 185  12 111  65 251 258 344 246 291
  221 282 373 301 178  83 201 296 121 166  77  66  96 345 106 244  56 348
  380 237  36 196 169 135 143  91 241  89 194 379 107 255 335 357 163 219
  208  71 171 340 209 160 328 352 321 230 314 184 382  53  35 173 249 119
  147 263 211 205 214 142 264 104  21 271 238 112 242 141 137  94  70  79
  191 180  62  76 315  59  37 101 257 297 188 346 152   9 179 324  23  61
   60   8 368 131 276  57 361 117 293 212 319  14 217 146 130 189  46 195
   33 223 132  22 199 236 170 339 133  10 126  43  13  88 250 279 124  95
  377 269 234 299 213 216 134 253 192 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi ' 'Never married']


['Male' 36 'Delhi' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 55 'Gurgaon' 'Separated, not legally divorced']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 41 'Bengaluru' 'Never married']


['Male' 35 'Delhi' 'Never married']


['Male' 50 'Bangalore' 'Widow/ Widower']


['Male' 40 'Bangalore ' 'Never married']


['Male' 40 'Hyderabad/pune' 

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.73617814  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.23772871  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.90232795  0.         ...  0.          0.
   0.        ]
 ...
 [ 1.          1.09146977  0.         ...  0.          0.
   0.        ]
 [ 0.          1.92221882  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.23772871  0.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']